In [ ]:
%load_ext autoreload
%autoreload 2

import os
from utils.config import Config
from featureanalysis.featureanalysis import parse_caller_feature
from utils.config import Config
from utils.viz import *
from utils.table import *
from utils.metrics import *
from utils.calltable import *
from utils.calltableseries import *
from utils.groundtruth import *
from utils.metricsseries import *
from utils.venn import venn6, get_labels
import random

import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_curve, f1_score, average_precision_score, precision_score, recall_score, mean_squared_error, accuracy_score, balanced_accuracy_score, roc_auc_score

import seaborn as sns 
import matplotlib.pyplot as plt

if not os.getcwd().endswith('cfdna_snv_benchmark'):
    os.chdir('../')
print('Current working directory: {}'.format(os.getcwd()))

config = Config("config/", "config_viz.yaml")

In [ ]:
# Chomosome

mixtureids =  ['CRC-1014_180816-CW-T_CRC-1014_090516-CW-T', 'CRC-986_100215-CW-T_CRC-986_300316-CW-T', 'CRC-123_310715-CW-T_CRC-123_121115-CW-T']
mixtureid = 'CRC-986_100215-CW-T_CRC-986_300316-CW-T'
#mixtureid = 'CRC-1014_180816-CW-T_CRC-1014_090516-CW-T'
#mixtureid = 'CRC-123_310715-CW-T_CRC-123_121115-CW-T'
reload = False
save = False
fixedvars=['coverage', 'ctdna']
filterparam = 'all'

markers = ['o', '^', 'X']
linestyles = ['-', '-', '-']
color_dict = {config.methods[i]: config.colors[i] for i in range(len(config.methods))}

muttypes = ['snv', 'indel']
metrics = ['auprc', 'precision', 'recall']

chrom = 'all'

print(config.methods)

In [ ]:
fixedvar = 'coverage'
if fixedvar == 'coverage':
    seriesorder = [(70, 0), (70, 80), (50, 100), (30, 120), (20, 130), (10, 140), (5, 145)]
    xaxis = 'tumor burden'
elif fixedvar == 'ctdna':
    seriesorder = [(70, 0), (70, 80), (70, 180)]
    xaxis = 'coverage'
print('############# {} ############'.format(mixtureid))
if mixtureid ==  'CRC-1014_180816-CW-T_CRC-1014_090516-CW-T':
    chroms = [str(c) for c in range(1,23) if c !=17 and c !=8 and c!=5 and c!=19 and c!=20 and c!=21 and c!=22]
elif mixtureid ==  'CRC-986_100215-CW-T_CRC-986_300316-CW-T':
    chroms = [str(c) for c in range(1,23) if c !=1 and c!= 2 and c !=8 and c!=20 and c!=21 and c!=22] 
else:
    chroms = [str(c) for c in range(1,23) if c !=1 and c!= 2 and c !=6 and c!=20 and c!=21] 

In [ ]:
fixedvar = 'coverage'
if fixedvar == 'coverage':
    seriesorder = [(70, 0), (70, 80), (50, 100), (30, 120), (20, 130), (10, 140), (5, 145)]
    xaxis = 'tumor burden'
elif fixedvar == 'ctdna':
    seriesorder = [(70, 0), (70, 80), (70, 180)]
    xaxis = 'coverage'
#for mixtureid in mixtureids:
print('############# {} ############'.format(mixtureid))
if mixtureid ==  'CRC-1014_180816-CW-T_CRC-1014_090516-CW-T':
    chroms = [str(c) for c in range(1,23) if c !=17 and c !=8 and c!=5 and c!=19 and c!=20 and c!=21 and c!=22]
elif mixtureid ==  'CRC-986_100215-CW-T_CRC-986_300316-CW-T':
    chroms = [str(c) for c in range(1,23) if c !=1 and c!= 2 and c !=8 and c!=20 and c!=21 and c!=22] 
else:
    chroms = [str(c) for c in range(1,23) if c !=1 and c!= 2 and c !=6 and c!=20 and c!=21] 
calltables = {'sampleid':[], 'tf':[], 'cov':[], 'snv':[], 'indel':[], 'snp':[]}
aux_all = []
calltable_snv, aux = get_calltableseries(config, mixtureid, chroms, muttype='snv', filterparam=filterparam, reload=reload, save=save)
calltable_indel, aux = get_calltableseries(config, mixtureid, chroms, muttype='indel', filterparam=filterparam, reload=reload, save=save)
calltable_snp, aux = get_calltableseries(config, mixtureid, chroms, muttype='snp', filterparam=filterparam, reload=reload, save=save)
print(calltable_snv.shape, calltable_indel.shape, calltable_snp.shape)
print(aux)
plasmasample = '_'.join(mixtureid.split('_')[:2])
print(plasmasample)
healthysample = '_'.join(mixtureid.split('_')[2:])
print(healthysample)
calltables['snv'].append(calltable_snv)
calltables['indel'].append(calltable_indel)
calltables['snp'].append(calltable_snp)
calltables['sampleid'] = mixtureid 
calltables['tf'] = np.unique([cn.split('_')[0] for cn in list(calltable_snv.columns)])[:-5].astype(float)
calltables['snv'] = pd.concat(calltables['snv'])
calltables['indel'] = pd.concat(calltables['indel'])
calltables['snp'] = pd.concat(calltables['snp'])
dilutionseries = aux.T[['mixture_' + '_'.join(mixtureid.split('_')[:2]) + '_' + str(s[0]) + 'x_' + '_'.join(mixtureid.split('_')[2:4]) + '_' + str(s[1]) + 'x' for s in seriesorder]].T
#for muttype in muttypes:
muttype = 'snv'
refsample = 'undiluted'
if muttype == 'snv':
    gtm = 5
else:  # elif muttype == 'indel':
    gtm = 4
print(max(aux['tf']))
#if mixtureid ==  'CRC-986_100215-CW-T_CRC-986_300316-CW-T':
#   gtm = 3
#    refsample = 'tissue'
#    calltablesseries  = generate_groundtruth(config, calltables[muttype], aux['tf'], ground_truth_method='tissue', muttype=muttype,
#                                        matchedtissuepath=os.path.join('data', 'matchedtissue', 'NCC_CRC-986_100215-T1W', 'calls', 'NCC_CRC-986_100215-T1W_snv_calls_PASS_exome.csv'))
#else:
#   calltablesseries = generate_groundtruth(config, calltables[muttype], aux['tf'], ground_truth_method=gtm, muttype=muttype)
calltablesseries = generate_groundtruth(config, calltables[muttype], aux['tf'], ground_truth_method=gtm, muttype=muttype,
                                    matchedtissuepath=None, methods=['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan', 'varnet', 'abemus', 'sinvict'])

In [ ]:
calltablesseries[['truth']]

In [ ]:
method = 'sinvict'
tflist = ['{:.2f}'.format(tf) for tf in dilutionseries.tf]
ressinvict = pd.DataFrame(columns=['precision', 'recall'])
filters = ['1. Poisson model', '2. Minimum Read Depth filter \n (conservative threshold: 5)', '3. Strand-bias filter\n (default: left=0.3, right=0.7)', '4. Average position of called location\n among all reads supporting the call',
           '5. Signal-to-Noise ratio filter', '6. Homopolymer Regions filter']
for thi, thresh in enumerate([4/6, 4/6, 2/6, 2/6, 5/6, 6/6]):
    aux = pd.DataFrame(calltablesseries[[tf+ '_'+method+'_score' for tf in tflist]].stack()).reset_index()
    aux.rename(columns={0: 'score'}, inplace=True)
    #aux.columns = ['chrom_pos_ref_alt', 'score']
    aux.set_index('chrom_pos_ref_alt', inplace=True)
    aux = aux.join(calltablesseries[['truth']], on='chrom_pos_ref_alt', how='outer')
    print(aux['truth'].value_counts())
    print(thresh)
    aux.loc[aux['score'] == thresh, 'score'] = True
    aux.loc[aux['score'] != True, 'score'] = False
    print(aux['score'].value_counts())
    print(aux['truth'].value_counts())
    aux = aux.astype(bool)
    print('precision: {:.2f}'.format(precision_score(aux['truth'], aux['score'])))
    print('recall: {:.2f}'.format(recall_score(aux['truth'], aux['score'])))
    ressinvict.at[filters[thi], 'precision'] = precision_score(aux['truth'], aux['score'])
    ressinvict.at[filters[thi], 'recall'] = recall_score(aux['truth'], aux['score'])

"""
    print(tf)
    for thresh in [4/6, 2/6]:
        print(thresh)
        aux = calltablesseries[[tf+ '_'+method +'_score', 'truth']]
        #print(aux[tf+ '_'+method +'_score'].value_counts())
        aux.loc[aux[tf+ '_'+method +'_score'] == thresh, tf+ '_'+method +'_score'] = True
        aux.loc[aux[tf+ '_'+method +'_score'] != True, tf+ '_'+method +'_score'] = False
        #print(aux[tf+ '_'+method +'_score'].value_counts())
        #print(aux['truth'].value_counts())
        aux = aux.astype(bool)
        print('precision: {:.2f}'.format(precision_score(aux['truth'], aux[tf+ '_'+method +'_score'])))
        print('recall: {:.2f}'.format(recall_score(aux['truth'], aux[tf+ '_'+method +'_score'])))
"""
ressinvict

In [ ]:
ressinvict.plot.bar(rot=90, color={"precision": "purple", "recall": "tab:pink"}, fontsize=20, figsize=(7,4))
plt.legend(fontsize=20)
plt.xlabel('SiNVICT successive filters',  fontsize=20)
plt.ylabel('score',  fontsize=20)
plt.ylim([0., 1])
plt.savefig('data/featureanalysis/featureimportance_'+method+'_patient986_150x.svg')
plt.savefig('data/featureanalysis/featureimportance_'+method+'_patient986_150x.png')

In [ ]:
np.random.seed(1234)
random.seed(42)
#for method in ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan', 'abemus']: #config.methods_tissue:
method = 'abemus'
color_dict = {config.methods[i]: config.colors[i] for i in range(len(config.methods))}
callmethod_snv_list = []
for mixtureid in mixtureids:
    #if mixtureid == 'CRC-986_100215-CW-T_CRC-986_300316-CW-T':
    for serie in [(70, 0), (70, 80), (50, 100), (30, 120), (20, 130), (10, 140), (5, 145)]:
        st, sn = serie
        #print(serie)
        if mixtureid ==  'CRC-1014_180816-CW-T_CRC-1014_090516-CW-T':
            chroms = [str(c) for c in range(1,23) if c !=17 and c !=8 and c!=5 and c!=19 and c!=20 and c!=21 and c!=22]
        elif mixtureid ==  'CRC-986_100215-CW-T_CRC-986_300316-CW-T':
            chroms = [str(c) for c in range(1,23) if c !=1 and c!= 2 and c !=8 and c!=20 and c!=21 and c!=22] 
        else:
            chroms = [str(c) for c in range(1,23) if c !=1 and c!= 2 and c !=6 and c!=20 and c!=21] 
        for chrom in chroms:
            #print(chrom)
            callmethod_snv, _, _ = parse_caller_feature(
                'data/mixtures/mixtures_chr'+chrom+'/mixtures_chr'+chrom+'_'+mixtureid+'/mixture_chr'+chrom+'_'+'_'.join(mixtureid.split('_')[:2])+'_'+str(st)+'x_'+'_'.join(mixtureid.split('_')[2:])+'_'+str(sn)+'x',
                method, save=False)
            if method in ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan']:
                callmethod_snv.drop(['ANN'], axis=1, inplace=True)
            #print(callmethod_snv.shape)
            #print(list(callmethod_snv.columns))
            #print(callmethod_snv.head())
            callmethod_snv_list.append(callmethod_snv)
callmethod_snv_allchrom = pd.concat(callmethod_snv_list)

# callmethod_snv_allchrom = pd.concat([callmethod_snv_allchrom, calltablesseries['truth']], axis=1)
callmethod_snv_allchrom = callmethod_snv_allchrom.join(calltablesseries[['truth']], how='outer')
print('hello', callmethod_snv_allchrom['truth'].isna().sum(), callmethod_snv_allchrom.shape[0])
callmethod_snv_allchrom['truth'].fillna(False, inplace=True)
callmethod_snv_allchrom['truth'].value_counts()
#callmethod_snv_allchrom.head()

features = list(callmethod_snv_allchrom.columns)
print(features)
for x in [method,  method+'_vaf', 'type', 'alt', 'chrom', 'pos', 'ref', method+'_totcov', method+'_altcov', 'truth', method+'_score']:
    print(x)
    features.remove(x)
print(features)

#callmethod_snv_allchrom = pd.concat([callmethod_snv_allchrom, calltablesseries['truth']], axis=1)
callmethod_snv_allchrom['truth'].fillna(False, inplace=True)
callmethod_snv_allchrom['truth'].value_counts()
callmethod_snv_allchrom[method].value_counts()
callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] == 'PASS'] = 1
callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] != 1] = 0

forbiden_features = ['OLD_VARIANT', 'TYPE', "cosmid_id", 'AN', "NS", 'CIGAR', 'technology.illumina', 'RUN', 'END_POS_REF', 'REF_MFVdVs', 'ALT_MFVdVs', 'Sample_Name'] #AF
score_features = ['ODDS', 'TLOD', 'SomaticEVS', 'SSF', 'SPV', 'pass.filter.pbem_coverage', 'filter.pbem_coverage', 'FALSE.']
#print(features)

Xlist = []
for feature in features:
    test = 0
    try:
        aux = callmethod_snv_allchrom[feature].fillna(0).astype(float)
        test = 1
    except:
        try: 
            aux = callmethod_snv_allchrom[feature].str.split(',').str[0].fillna(0).astype(float)
            test = 1
        except:
            try:
                aux = callmethod_snv_allchrom[feature]
                aux[aux == '.'] = 'nan'
                aux.fillna(0, inplace=True)
                aux = aux.astype(float)
                test =1
            except:
                test = 0
                print(feature, 'no')
    if test == 1 and feature not in forbiden_features and feature not in score_features:
        print(feature, 'yes')
        aux = aux.astype(float)
        aux.fillna(0, inplace=True)
        Xlist.append(aux)

X = pd.concat(Xlist, axis=1)
print(X.columns)
if method == 'vardict':
    X['SOR'][X['SOR'] == np.inf] = X['SOR'][X['SOR'] != np.inf].max()  # replace inf values
y = callmethod_snv_allchrom[method+'_score'].fillna(0)
plt.figure()
y.plot.hist()
plt.show()
print(X.shape, y.shape)

print(y.describe())

feature_name_dict = {}
for i, x in enumerate(list(X.columns)):
    feature_name_dict[i] = x
feature_name_dict

#model = DecisionTreeRegressor(random_state=0, max_depth=3)
model = RandomForestRegressor(random_state=0)
#grid = {
#        'bootstrap': [True, False],
#        'max_depth':  [10, 25, 50, 75, 100, 200, None], #[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#        'max_features': ['auto', 'sqrt'],
#        'min_samples_leaf': [1, 2, 4],
#        'min_samples_split': [2, 5, 10],
#        'n_estimators': [200, 400, 600, 800, 1000] #, 1200, 1400, 1600, 1800, 2000]
#   }
#model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False})
#if method == 'strelka2':
#    model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False})
#elif method == 'mutect2':
#    model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False})
#elif method == 'abemus':
#    model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False}) # abemus
#else:
#gridsearch = RandomizedSearchCV(estimator=model, param_distributions=grid, n_jobs=8, scoring="roc_auc", n_iter=25, cv=5, verbose=6, random_state=0)
#gridsearch.fit(X, y)
#print(gridsearch.best_params_)
# save best params
#pd.Series(gridsearch.best_params_).to_csv('data/featureanalysis/bestparams_'+method+'_986_150x.csv')
#model = model.set_params(**gridsearch.best_params_) # retrive best hyper params
# save model
#import pickle
#with open("data/featureanalysis/model_"+method+"_986_150x.pkl", "wb") as f:
#    pickle.dump(model, f)
#cv = StratifiedKFold(n_splits=10, shuffle = True, random_state=0) # prepare 10-fold CV
#for i, cvi in enumerate(cv):
cvn = 10
sfolder = KFold(n_splits=cvn,random_state=0,shuffle=False)
cv_results = cross_validate(model, X, y, cv=sfolder, scoring='neg_mean_squared_error', return_estimator=True, verbose=6)
print(cv_results['test_score'])
print(y[y!=0].describe())
# save cv results
#pd.DataFrame(cv_results).to_csv('data/featureanalysis/cvresults_'+method+'_3patients_150x.csv')

#model.fit(X, y)
#ypred = model.predict(X)
#print(sklearn.metrics.mean_squared_error(y, ypred))
plt.rc('font', size=20) 
fi_pd_list = []
rocauclist = []
for idx, estimator in enumerate(cv_results['estimator']):
    for ci, (trainindex, testindex) in enumerate(sfolder.split(X,y)):
        if ci == c:
            print(X.shape)
            ypredi = pd.Series(estimator.predict([X.iloc[i,:] for i in testindex]))
            yi = pd.Series([y[i] for i in testindex])
            #print(cv_results['test_score'][idx], mean_squared_error(yi, ypredi))
            thrs = 0.5
            if method == 'smurf':
                thrs = 0.0822
            yi[yi >= thrs ] = 1
            yi[yi < thrs ] = 0
            ypredi[ypredi >= thrs ] = 1
            ypredi[ypredi < thrs ] = 0
            print('F1: ', f1_score(yi, ypredi))
            print('ROC AUC: ', roc_auc_score(yi, ypredi))
            print('Balanced accuracy: ', balanced_accuracy_score(yi, ypredi))
            print('Accuracy: ', accuracy_score(yi, ypredi))
            rocauclist.append(roc_auc_score(yi, ypredi))
    c += 1
    fi_pd = pd.DataFrame(estimator.feature_importances_,
                         columns=['importance']).sort_values('importance', ascending=False)
    fi_pd_list.append(fi_pd)
print('ROC AUC = {:.2f} +/- {:.2f}'.format(np.mean(rocauclist), np.std(rocauclist)))
feature_importances = pd.concat(fi_pd_list, axis=1)
feature_importances.columns = ['estimator_'+str(i) for i in range(cvn)]
feature_importances.index = feature_name_dict.values()
feature_importances['median'] = feature_importances.median(axis=1)
#feature_importances.to_csv('data/featureanalysis/featureimportance_df_'+method+'_3patients_150x.csv')
feature_importances = feature_importances[(feature_importances.T != 0).any()]
feature_importances = feature_importances[(feature_importances['median'] > 0.01)]
feature_importances.sort_values(by='median', ascending=False, inplace=True)
g = sns.catplot(data=feature_importances.T.drop('median'), kind="box", color=color_dict[method],
                height=6, aspect=3.5);
g.map_dataframe(sns.stripplot,
                palette=["k"], dodge=True)
g.set_xticklabels(rotation=45, ha='right')
plt.xlabel('Features')
plt.ylabel('Feature importance in Random Forest models in CV')
plt.title(method + ' feature importance analysis. CV MSE = {:.2E} +/- {:.2E}'.format(-np.mean(cv_results['test_score']), np.std(cv_results['test_score'])))
#plt.savefig('data/featureanalysis/featureimportance_'+method+'_3patients_150x.svg')
#plt.savefig('data/featureanalysis/featureimportance_'+method+'_3patients_150x.png')
plt.show()

#from sklearn.tree import export_graphviz
#import graphviz#

#dot_data = export_graphviz(model, feature_names=X.columns, filled=True, rounded=True)  

#graph = graphviz.Source(dot_data)
#graph.render("tree_"+method) 

In [ ]:
#model = RandomForestRegressor(random_state=0)
#cvn = 10
#sfolder = KFold(n_splits=cvn,random_state=0,shuffle=False)
#cv_results = cross_validate(model, X, y, cv=sfolder, scoring='neg_mean_squared_error', return_estimator=True, verbose=6)
#print(cv_results['test_score'])
#print(y[y!=0].describe())
# save cv results
#pd.DataFrame(cv_results).to_csv('data/featureanalysis/cvresults_'+method+'_3patients_150x.csv')

#model.fit(X, y)
#ypred = model.predict(X)
#print(sklearn.metrics.mean_squared_error(y, ypred))
#y = y.values
plt.rc('font', size=20) 
fi_pd_list = []
c = 0
rocauclist = []
for idx, estimator in enumerate(cv_results['estimator']):
    for ci, (trainindex, testindex) in enumerate(sfolder.split(X,y)):
        if ci == c:
            print(X.shape)
            ypredi = pd.Series(estimator.predict([X.iloc[i,:] for i in testindex]))
            yi = pd.Series([y[i] for i in testindex])
            #print(cv_results['test_score'][idx], mean_squared_error(yi, ypredi))
            threshold = 0.5
            if method == 'abemus':
                threshold = 0.5
            yi[yi >= threshold ] = 1
            yi[yi < threshold ] = 0
            ypredi[ypredi >= threshold ] = 1
            ypredi[ypredi < threshold ] = 0
            print('F1: ', f1_score(yi, ypredi))
            try:
                print('ROC AUC: ', roc_auc_score(yi, ypredi))
                rocauclist.append(roc_auc_score(yi, ypredi))
            except:
                rocauclist.append(np.nan)
            print('Balanced accuracy: ', balanced_accuracy_score(yi, ypredi))
            print('Accuracy: ', accuracy_score(yi, ypredi))
            print(yi.sum()/yi.shape[0])
            
    c += 1
    fi_pd = pd.DataFrame(estimator.feature_importances_,
                         columns=['importance']).sort_values('importance', ascending=False)
    fi_pd_list.append(fi_pd)
print('ROC AUC = {:.2f} +/- {:.2f}'.format(np.mean(rocauclist), np.std(rocauclist)))

In [ ]:
rocauclist

In [ ]:
#calltablesseries = pd.read_csv(os.path.join('data', 'mixtures_ultradeep', 'mixtures_chrall', 'mixtures_chrall_CRC-986_100215-CW-T_CRC-986_300316-CW-T' , 'calls', 'CRC-986_100215-CW-T_CRC-986_300316-CW-T_snv_calls_all.csv'), index_col=0, memory_map=True)
#aux = pd.read_csv(os.path.join('data', 'mixtures_ultradeep', 'mixtures_chrall', 'mixtures_chrall_CRC-986_100215-CW-T_CRC-986_300316-CW-T' , 'calls', 'CRC-986_100215-CW-T_CRC-986_300316-CW-T_tf_cov.csv'),index_col=0)
gtm = 5
muttype = 'snv'
refsample = 'undiluted'
fixedvar = 'ctdna'
if fixedvar == 'coverage':
    seriesorder = [(1000, 0), (1000, 1000), (750, 1250), (500, 1500), (400, 1600), (200, 1800), (100, 1900)] 
    xaxis = 'tumor burden'
elif fixedvar == 'ctdna':
    seriesorder = [(1000, 0), (1000, 1000), (1000, 1900)]
    xaxis = 'coverage'
mixtureid = 'CRC-986_100215-CW-T_CRC-986_300316-CW-T'
#dilutionseries = aux.T[['mixture_chrall_' + '_'.join(mixtureid.split('_')[:2]) + '_' + str(s[0]) + 'x_' + '_'.join(mixtureid.split('_')[2:4]) + '_' + str(s[1]) + 'x' for s in seriesorder]].T
#calltablesseries = generate_groundtruth(config, calltablesseries, aux['tf'], ground_truth_method=gtm, muttype=muttype,
#                                       matchedtissuepath=None, methods=['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan', 'varnet', 'abemus', 'sinvict'])
np.random.seed(1234)
random.seed(42)
#for method in ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan', 'abemus']: #config.methods_tissue:
method = 'freebayes'
color_dict = {config.methods[i]: config.colors[i] for i in range(len(config.methods))}
callmethod_snv_list = []
for mixtureid in mixtureids:
    if mixtureid == 'CRC-986_100215-CW-T_CRC-986_300316-CW-T':
        for serie in [(1000, 0), (1000, 1000), (750, 1250), (500, 1500), (400, 1600), (200, 1800), (100, 1900)]:
            st, sn = serie
            #print(serie)
            #if mixtureid ==  'CRC-1014_180816-CW-T_CRC-1014_090516-CW-T':
            #    chroms = [str(c) for c in range(1,23) if c !=17 and c !=8 and c!=5 and c!=19 and c!=20 and c!=21 and c!=22]
            #elif mixtureid ==  'CRC-986_100215-CW-T_CRC-986_300316-CW-T':
            #    chroms = [str(c) for c in range(1,23) if c !=1 and c!= 2 and c !=8 and c!=20 and c!=21 and c!=22] 
            #else:
            #    chroms = [str(c) for c in range(1,23) if c !=1 and c!= 2 and c !=6 and c!=20 and c!=21] 
            #for chrom in chroms:
                #print(chrom)
            chrom = 'all'
            callmethod_snv, _, _ = parse_caller_feature(
                'data/mixtures_ultradeep/mixtures_chr'+chrom+'/mixtures_chr'+chrom+'_'+mixtureid+'/mixture_chr'+chrom+'_'+'_'.join(mixtureid.split('_')[:2])+'_'+str(st)+'x_'+'_'.join(mixtureid.split('_')[2:])+'_'+str(sn)+'x',
                method, save=False)
            if method in ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan']:
                callmethod_snv.drop(['ANN'], axis=1, inplace=True)
            #print(callmethod_snv.shape)
            #print(list(callmethod_snv.columns))
            #print(callmethod_snv.head())
            callmethod_snv_list.append(callmethod_snv)
callmethod_snv_allchrom = pd.concat(callmethod_snv_list)

# callmethod_snv_allchrom = pd.concat([callmethod_snv_allchrom, calltablesseries['truth']], axis=1)
callmethod_snv_allchrom = callmethod_snv_allchrom.join(calltablesseries[['truth']], how='outer')
callmethod_snv_allchrom['truth'].fillna(False, inplace=True)
callmethod_snv_allchrom['truth'].value_counts()
#callmethod_snv_allchrom.head()

features = list(callmethod_snv_allchrom.columns)
print(features)
for x in [method,  method+'_vaf', 'type', 'alt', 'chrom', 'pos', 'ref', method+'_totcov', method+'_altcov', 'truth', method+'_score']:
    print(x)
    features.remove(x)
print(features)

#callmethod_snv_allchrom = pd.concat([callmethod_snv_allchrom, calltablesseries['truth']], axis=1)
callmethod_snv_allchrom['truth'].fillna(False, inplace=True)
callmethod_snv_allchrom['truth'].value_counts()

callmethod_snv_allchrom[method].value_counts()
callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] == 'PASS'] = 1
callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] != 1] = 0

forbiden_features = ['OLD_VARIANT', 'TYPE', "cosmid_id", 'AN', "NS", 'CIGAR', 'technology.illumina', 'RUN'] #AF
score_features = ['ODDS', 'TLOD', 'SomaticEVS', 'SSF', 'SPV', 'pass.filter.pbem_coverage', 'filter.pbem_coverage']
print(features)

Xlist = []
for feature in features:
    # print(feature)
    test = 0
    try:
        aux = callmethod_snv_allchrom[feature].fillna(0).astype(float)
        test = 1
    except:
        try: 
            aux = callmethod_snv_allchrom[feature].str.split(',').str[0].fillna(0).astype(float)
            test = 1
        except:    
            test = 0
    if test == 1 and feature not in forbiden_features and feature not in score_features:
        print(feature)
        Xlist.append(aux)
X = pd.concat(Xlist, axis=1)
if method == 'vardict':
    X['SOR'][X['SOR'] == np.inf] = X['SOR'][X['SOR'] != np.inf].max()  # replace inf values
y = callmethod_snv_allchrom[method+'_score'].fillna(0)
plt.figure()
y.plot.hist()
plt.show()
print(X.shape, y.shape)

print(y.describe())

feature_name_dict = {}
for i, x in enumerate(list(X.columns)):
    feature_name_dict[i] = x
feature_name_dict

#model = DecisionTreeRegressor(random_state=0, max_depth=3)
model = RandomForestRegressor(random_state=0)
#grid = {
#        'bootstrap': [True, False],
#        'max_depth':  [10, 25, 50, 75, 100, 200, None], #[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#        'max_features': ['auto', 'sqrt'],
#        'min_samples_leaf': [1, 2, 4],
#        'min_samples_split': [2, 5, 10],
#        'n_estimators': [200, 400, 600, 800, 1000] #, 1200, 1400, 1600, 1800, 2000]
#   }
#model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False})
#if method == 'strelka2':
#    model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False})
#elif method == 'mutect2':
#    model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False})
#elif method == 'abemus':
#    model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False}) # abemus
#else:
#gridsearch = RandomizedSearchCV(estimator=model, param_distributions=grid, n_jobs=8, scoring="roc_auc", n_iter=25, cv=5, verbose=6, random_state=0)
#gridsearch.fit(X, y)
#print(gridsearch.best_params_)
# save best params
#pd.Series(gridsearch.best_params_).to_csv('data/featureanalysis/bestparams_'+method+'_986_150x.csv')
#model = model.set_params(**gridsearch.best_params_) # retrive best hyper params
# save model
#import pickle
#with open("data/featureanalysis/model_"+method+"_986_150x.pkl", "wb") as f:
#    pickle.dump(model, f)
#cv = StratifiedKFold(n_splits=10, shuffle = True, random_state=0) # prepare 10-fold CV
#for i, cvi in enumerate(cv):   
cvn = 10
cv_results = cross_validate(model, X, y, cv=cvn, scoring='neg_mean_squared_error', return_estimator=True, verbose=6)
print(cv_results['test_score'])
print(y[y!=0].describe())
# save cv results
pd.DataFrame(cv_results).to_csv('data/featureanalysis/cvresults_'+method+'_patient986_2000x.csv')

#model.fit(X, y)
#ypred = model.predict(X)
#print(sklearn.metrics.mean_squared_error(y, ypred))
plt.rc('font', size=20) 
fi_pd_list = []
for idx, estimator in enumerate(cv_results['estimator']):
    fi_pd = pd.DataFrame(estimator.feature_importances_,
                         columns=['importance']).sort_values('importance', ascending=False)
    fi_pd_list.append(fi_pd)
feature_importances = pd.concat(fi_pd_list, axis=1)
feature_importances.columns = ['estimator_'+str(i) for i in range(cvn)]
feature_importances.index = feature_name_dict.values()
feature_importances['median'] = feature_importances.median(axis=1)
feature_importances.to_csv('data/featureanalysis/featureimportance_df_'+method+'_patient986_2000x.csv')
feature_importances = feature_importances[(feature_importances.T != 0).any()]
#feature_importances = feature_importances[(feature_importances['median'] > 0.01)]
feature_importances.sort_values(by='median', ascending=False, inplace=True)
g = sns.catplot(data=feature_importances.T.drop('median'), kind="box", color=color_dict[method],
                height=6, aspect=3.5);
g.map_dataframe(sns.stripplot,
                palette=["k"], dodge=True)
g.set_xticklabels(rotation=45, ha='right')
plt.xlabel('Features')
plt.ylabel('Feature importance in Random Forest models in CV')
plt.title(method + ' feature importance analysis. CV MSE = {:.2E} +/- {:.2E}'.format(-np.mean(cv_results['test_score']), np.std(cv_results['test_score'])))
plt.savefig('data/featureanalysis/featureimportance_'+method+'_patient986_2000x.svg')
plt.savefig('data/featureanalysis/featureimportance_'+method+'_patient986_2000x.png')
plt.show()

#from sklearn.tree import export_graphviz
#import graphviz#

#dot_data = export_graphviz(model, feature_names=X.columns, filled=True, rounded=True)  

#graph = graphviz.Source(dot_data)
#graph.render("tree_"+method) 


In [ ]:
feature_importances.sort_values(by='median', ascending=False)

In [ ]:
tabledict = pd.DataFrame()
colorlist = []
for method in config.methods:
    if method != 'varnet' and method != 'sinvict':
        feature_importances = pd.read_csv('data/featureanalysis/featureimportance_df_'+method+'_3patients_150x.csv', index_col=0)
        feature_importances = feature_importances[(feature_importances.T != 0).any()]
        feature_importances.sort_values(by='median', ascending=False, inplace=True)
        feature_importances.drop('median', axis=1, inplace=True)
        if method == 'mutect2':
            feature_importances = feature_importances.T[['ECNT', 'MQ', 'DP', 'NLOD', 'NALOD']].T
        feature_importances = feature_importances.iloc[:5, :]
        feature_importances = feature_importances.stack().reset_index().drop('level_1', axis=1)
        feature_importances.rename(columns={'level_0': 'feature', 0: 'importance'}, inplace=True)
        cvresults = pd.read_csv('data/featureanalysis/cvresults_'+method+'_3patients_150x.csv', index_col=0)
        feature_importances['caller'] = method + ' \n CV MSE = {:.2E} +/- {:.2E}'.format(-np.mean(cvresults['test_score']), np.std(cvresults['test_score']))
        #feature_importances.index = [l +'_'+method for l in feature_importances.index]
        #feature_importances.reset_index(inplace=True)
        #feature_importances.rename(columns={'index': 'feature'}, inplace=True)
        tabledict = pd.concat([tabledict, feature_importances], axis=0)
        colorlist.append(config.colors[config.methods.index(method)])
        
print(colorlist)

dictcol = {}
for method in config.methods:
    if method != 'varnet' and method != 'sinvict':
        cvresults = pd.read_csv('data/featureanalysis/cvresults_'+method+'_3patients_150x.csv', index_col=0)
        dictcol[method + ' \n CV MSE = {:.2E} +/- {:.2E}'.format(-np.mean(cvresults['test_score']), np.std(cvresults['test_score']))] = config.colors[config.methods.index(method)]
dictcol
#g = sns.catplot(data=tabledict.T, kind="box", color=colorlist, height=6, aspect=3.5);
#g.map_dataframe(sns.stripplot, palette=["k"], dodge=True)
#g.set_xticklabels(rotation=45, ha='right')
#plt.xlabel('Features')
#plt.ylabel('Feature importance in Random Forest models in CV')
#plt.title('Feature importance analysis')
#sns.set(font_scale=2)
sns.reset_defaults()
sns.set(style="white", font_scale=1.5)
g = sns.FacetGrid(tabledict, col='caller', sharex=False, col_wrap=3, height=6, aspect=1)
g.map_dataframe(sns.boxplot, x="feature", y="importance", hue='caller', palette=dictcol, fliersize=0)
g.map_dataframe(sns.stripplot, x='feature', y='importance', palette=["k"], dodge=True)

for ax in g.axes:
    ax.axhline(0.1, ls='-', c='k')
    ax.axhline(0.01, ls='--', c='k')
    
plt.savefig('data/featureanalysis/figure5_featureimportance_3patients_150x.svg')
plt.savefig('data/featureanalysis/figure5_featureimportance_3patients_150x.png')

In [ ]:
tabledict = pd.DataFrame()
colorlist = []
method = 'abemus'
for cov in ['3patients_150x', 'patient986_2000x']:
    feature_importances = pd.read_csv('data/featureanalysis/featureimportance_df_'+method+'_'+cov+'.csv', index_col=0)
    feature_importances = feature_importances[(feature_importances.T != 0).any()]
    feature_importances.sort_values(by='median', ascending=False, inplace=True)
    #feature_importances.drop('median', axis=1, inplace=True)
    feature_importances = feature_importances[['median']]
    feature_importances = feature_importances.stack().reset_index().drop('level_1', axis=1)
    feature_importances.rename(columns={'level_0': 'feature', 0: 'importance_'+cov.split('_')[1]}, inplace=True)
    feature_importances.set_index('feature', inplace=True)
    cvresults = pd.read_csv('data/featureanalysis/cvresults_'+method+'_'+cov+'.csv', index_col=0)
    #feature_importances['coverage'] = cov # + ' \n CV MSE = {:.2E} +/- {:.2E}'.format(-np.mean(cvresults['test_score']), np.std(cvresults['test_score']))
    #feature_importances.index = [l +'_'+method for l in feature_importances.index]
    #feature_importances.reset_index(inplace=True)
    #feature_importances.rename(columns={'index': 'feature'}, inplace=True)
    tabledict = pd.concat([tabledict, feature_importances], axis=1)
    colorlist.append(config.colors[config.methods.index(method)])
        
print(colorlist)
listfeat = set(set(list(tabledict['importance_150x'].iloc[:5].index)) | set(list(tabledict['importance_2000x'].sort_values(ascending=False).iloc[:5].index)))
#g = sns.catplot(data=tabledict.T, kind="box", color=colorlist, height=6, aspect=3.5);
#g.map_dataframe(sns.stripplot, palette=["k"], dodge=True)
#g.set_xticklabels(rotation=45, ha='right')
#plt.xlabel('Features')
#plt.ylabel('Feature importance in Random Forest models in CV')
#plt.title('Feature importance analysis')
#sns.set(font_scale=2)
#sns.reset_defaults()
#sns.set(style="white", font_scale=1.5)
#g = sns.FacetGrid(tabledict, col='coverage', sharex=False, col_wrap=3, height=6, aspect=1)
#g.map_dataframe(sns.boxplot, x="feature", y="importance", hue='coverage', palette=dictcol, fliersize=0)
#g.map_dataframe(sns.stripplot, x='feature', y='importance', palette=["k"], dodge=True)
sns.scatterplot(x='importance_150x', y='importance_2000x', data=tabledict.loc[listfeat].reset_index(), style='feature', linewidth=2, s=300, fc="none", ec='tab:brown')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.plot([0,1],[0,1], ls='--', c='k')
#plt.show()
#for ax in g.axes:
#    ax.axhline(0.1, ls='-', c='k')
#    ax.axhline(0.01, ls='--', c='k')
    
plt.savefig('data/featureanalysis/figure5_abemus_150xvs2000x.svg')
plt.savefig('data/featureanalysis/figure5_abemus_150xvs2000x.png')

In [ ]:
colorlist[2]

In [ ]:
tabledict

In [ ]:
X['SOR'][X['SOR'] != np.inf].max()

In [ ]:
np.random.seed(1234)
#for method in ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan', 'abemus']: #config.methods_tissue:
method = 'abemus'
color_dict = {config.methods[i]: config.colors[i] for i in range(len(config.methods))}
callmethod_snv_list = []
for mixtureid in mixtureids:
    for serie in [(70, 0), (70, 80), (50, 100), (30, 120), (20, 130), (10, 140), (5, 145)]:
        st, sn = serie
        #print(serie)
        if mixtureid ==  'CRC-1014_180816-CW-T_CRC-1014_090516-CW-T':
            chroms = [str(c) for c in range(1,23) if c !=17 and c !=8 and c!=5 and c!=19 and c!=20 and c!=21 and c!=22]
        elif mixtureid ==  'CRC-986_100215-CW-T_CRC-986_300316-CW-T':
            chroms = [str(c) for c in range(1,23) if c !=1 and c!= 2 and c !=8 and c!=20 and c!=21 and c!=22] 
        else:
            chroms = [str(c) for c in range(1,23) if c !=1 and c!= 2 and c !=6 and c!=20 and c!=21] 
        for chrom in chroms:
            #print(chrom)
            callmethod_snv, _, _ = parse_caller_feature(
                'data/mixtures/mixtures_chr'+chrom+'/mixtures_chr'+chrom+'_'+mixtureid+'/mixture_chr'+chrom+'_'+'_'.join(mixtureid.split('_')[:2])+'_'+str(st)+'x_'+'_'.join(mixtureid.split('_')[2:])+'_'+str(sn)+'x',
                method, save=False)
            if method in ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan']:
                callmethod_snv.drop(['ANN'], axis=1, inplace=True)
            #print(callmethod_snv.shape)
            #print(list(callmethod_snv.columns))
            #print(callmethod_snv.head())
            callmethod_snv_list.append(callmethod_snv)
callmethod_snv_allchrom = pd.concat(callmethod_snv_list)

# callmethod_snv_allchrom = pd.concat([callmethod_snv_allchrom, calltablesseries['truth']], axis=1)
callmethod_snv_allchrom = callmethod_snv_allchrom.join(calltablesseries[['truth']], how='outer')
callmethod_snv_allchrom['truth'].fillna(False, inplace=True)
callmethod_snv_allchrom['truth'].value_counts()
#callmethod_snv_allchrom.head()

features = list(callmethod_snv_allchrom.columns)
print(features)
for x in [method,  method+'_vaf', 'type', 'alt', 'chrom', 'pos', 'ref', method+'_totcov', method+'_altcov', 'truth', method+'_score']:
    print(x)
    features.remove(x)
print(features)

#callmethod_snv_allchrom = pd.concat([callmethod_snv_allchrom, calltablesseries['truth']], axis=1)
callmethod_snv_allchrom['truth'].fillna(False, inplace=True)
callmethod_snv_allchrom['truth'].value_counts()

callmethod_snv_allchrom[method].value_counts()
callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] == 'PASS'] = 1
callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] != 1] = 0

forbiden_features = ['OLD_VARIANT', 'TYPE', "cosmid_id", 'AN', "NS", 'CIGAR', 'technology.illumina', 'RUN'] #AF
score_features = ['ODDS', 'TLOD', 'SomaticEVS', 'SSF', 'SPV', 'pass.filter.pbem_coverage', 'filter.pbem_coverage']
print(features)

Xlist = []
for feature in features:
    # print(feature)
    test = 0
    try:
        aux = callmethod_snv_allchrom[feature].fillna(0).astype(float)
        test = 1
    except:
        try: 
            aux = callmethod_snv_allchrom[feature].str.split(',').str[0].fillna(0).astype(float)
            test = 1
        except:    
            test = 0
    if test == 1 and feature not in forbiden_features and feature not in score_features:
        print(feature)
        Xlist.append(aux)
X = pd.concat(Xlist, axis=1)
y = callmethod_snv_allchrom[method+'_score'].fillna(0)
plt.figure()
y.plot.hist()
plt.show()
print(X.shape, y.shape)

print(y.describe())

feature_name_dict = {}
for i, x in enumerate(list(X.columns)):
    feature_name_dict[i] = x
feature_name_dict

model = DecisionTreeRegressor(random_state=0, max_depth=4)
#model = RandomForestRegressor(random_state=0)
#grid = {
#        'bootstrap': [True, False],
#        'max_depth':  [10, 25, 50, 75, 100, 200, None], #[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#        'max_features': ['auto', 'sqrt'],
#        'min_samples_leaf': [1, 2, 4],
#        'min_samples_split': [2, 5, 10],
#        'n_estimators': [200, 400, 600, 800, 1000] #, 1200, 1400, 1600, 1800, 2000]
#   }
#model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False})
#if method == 'strelka2':
#    model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False})
#elif method == 'mutect2':
#    model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False})
#elif method == 'abemus':
#    model = model.set_params(**{'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False}) # abemus
#else:
#gridsearch = RandomizedSearchCV(estimator=model, param_distributions=grid, n_jobs=8, scoring="roc_auc", n_iter=25, cv=5, verbose=6, random_state=0)
#gridsearch.fit(X, y)
#print(gridsearch.best_params_)
# save best params
#pd.Series(gridsearch.best_params_).to_csv('data/featureanalysis/bestparams_'+method+'_986_150x.csv')
#model = model.set_params(**gridsearch.best_params_) # retrive best hyper params
# save model
#import pickle
#with open("data/featureanalysis/model_"+method+"_986_150x.pkl", "wb") as f:
#    pickle.dump(model, f)
#cv = StratifiedKFold(n_splits=10, shuffle = True, random_state=0) # prepare 10-fold CV
#for i, cvi in enumerate(cv):   
cv_results = cross_validate(model, X, y, cv=10, scoring='neg_mean_squared_error', return_estimator=True, verbose=6)
print(cv_results['test_score'])
print(y[y!=0].describe())
# save cv results
pd.DataFrame(cv_results).to_csv('data/featureanalysis/cvresults_'+method+'_3patients_150x.csv')

model.fit(X, y)
ypred = model.predict(X)
print(sklearn.metrics.mean_squared_error(y, ypred))

from sklearn.tree import export_graphviz
import graphviz#

dot_data = export_graphviz(model, feature_names=X.columns, filled=True, rounded=True)  

graph = graphviz.Source(dot_data)
graph.render("data/featureanalysis/tree_"+method+'_3patients') 

In [ ]:
print('{:.2E}'.format(sklearn.metrics.mean_squared_error(y, ypred)))
y.describe()
y.describe()

In [ ]:
sorted(sklearn.metrics.SCORERS.keys())

In [ ]:
plt.rc('font', size=20) 
fi_pd_list = []
for idx, estimator in enumerate(cv_results['estimator']):
    fi_pd = pd.DataFrame(estimator.feature_importances_,
                         columns=['importance']).sort_values('importance', ascending=False)
    fi_pd_list.append(fi_pd)
feature_importances = pd.concat(fi_pd_list, axis=1)
feature_importances.columns = ['estimator_'+str(i) for i in range(10)]
feature_importances.index = feature_name_dict.values()
feature_importances['median'] = feature_importances.median(axis=1)
feature_importances = feature_importances[(feature_importances.T != 0).any()]
#feature_importances = feature_importances[(feature_importances['median'] > 0.01)]
feature_importances.sort_values(by='median', ascending=False, inplace=True)
g = sns.catplot(data=feature_importances.T.drop('median'), kind="box", color=color_dict[method],
                height=6, aspect=3.5);
g.map_dataframe(sns.stripplot,
                palette=["k"], dodge=True)
g.set_xticklabels(rotation=45, ha='right')
plt.xlabel('Features')
plt.ylabel('Feature importance in Random Forest models in CV')
plt.title(method + ' feature importance analysis. CV MSE = {:.2E} +/- {:.2E}'.format(-np.mean(cv_results['test_score']), np.std(cv_results['test_score'])))
plt.show()

In [ ]:
plt.rc('font', size=20) 
fi_pd_list = []
for idx, estimator in enumerate(cv_results['estimator']):
    fi_pd = pd.DataFrame(estimator.feature_importances_,
                         columns=['importance']).sort_values('importance', ascending=False)
    fi_pd_list.append(fi_pd)
feature_importances = pd.concat(fi_pd_list, axis=1)
feature_importances.columns = ['estimator_'+str(i) for i in range(3)]
feature_importances.index = feature_name_dict.values()
feature_importances['median'] = feature_importances.median(axis=1)
feature_importances = feature_importances[(feature_importances.T != 0).any()]
feature_importances = feature_importances[(feature_importances['median'] > 0.01)]
feature_importances.sort_values(by='median', ascending=False, inplace=True)
g = sns.catplot(data=feature_importances.T.drop('median'), kind="box", color=color_dict[method],
                height=6, aspect=3.5);
g.map_dataframe(sns.stripplot,
                palette=["k"], dodge=True)
g.set_xticklabels(rotation=45, ha='right')
plt.xlabel('Features')
plt.ylabel('Feature importance in Random Forest models in CV')
plt.title(method + ' feature importance analysis. CV MSE (acc {:.2f} +/- {:.2f})'.format(np.mean(cv_results['test_score']), np.std(cv_results['test_score'])))
#plt.savefig('data/featureanalysis/featureimportance_'+method+'_986_150x.svg')
#plt.savefig('data/featureanalysis/featureimportance_'+method+'_986_150x.png')
plt.show()

In [ ]:
feature_importances.median(axis=1)

In [ ]:
plt.rc('font', size=20) 
fi_pd_list = []
for idx, estimator in enumerate(cv_results['estimator']):
    fi_pd = pd.DataFrame(estimator.feature_importances_,
                         columns=['importance']).sort_values('importance', ascending=False)
    fi_pd_list.append(fi_pd)
feature_importances = pd.concat(fi_pd_list, axis=1)
feature_importances.columns = ['estimator_'+str(i) for i in range(3)]
feature_importances.index = feature_name_dict.values()
feature_importances['median'] = feature_importances.median(axis=1)
feature_importances.sort_values(by='estimator_0', ascending=False, inplace=True)
feature_importances = feature_importances[(feature_importances.T != 0).any()]
feature_importances = feature_importances[(feature_importances['median'] > 0.01)]
g = sns.catplot(data=feature_importances.T, kind="box", color=color_dict[method],
                height=6, aspect=3.5);
g.map_dataframe(sns.stripplot,
                palette=["k"], dodge=True)
g.set_xticklabels(rotation=45, ha='right')
plt.xlabel('Features')
plt.ylabel('Feature importance in Random Forest models in CV')
plt.title(method + ' feature importance analysis. CV MSE (acc {:.2f} +/- {:.2f})'.format(np.mean(cv_results['test_score']), np.std(cv_results['test_score'])))
#plt.savefig('data/featureanalysis/featureimportance_'+method+'_986_150x.svg')
#plt.savefig('data/featureanalysis/featureimportance_'+method+'_986_150x.png')
plt.show()

In [ ]:

from sklearn.tree import export_graphviz
import graphviz

dot_data = export_graphviz(model, feature_names=X.columns, filled=True, rounded=True)  

graph = graphviz.Source(dot_data)
graph.render("tree") 

In [ ]:
np.random.seed(1234)
feature_importances_all = []
for method in ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan', 'abemus']:
    color_dict = {config.methods[i]: config.colors[i] for i in range(len(config.methods))}
    callmethod_snv_list = []
    for serie in [(70, 0), (70, 80), (50, 100), (30, 120), (20, 130), (10, 140), (5, 145)]:
        st, sn = serie
        #print(serie)
        for chrom in chroms:
            #print(chrom)
            callmethod_snv, _, _ = parse_caller_feature(
                'data/mixtures/mixtures_chr'+chrom+'/mixtures_chr'+chrom+'_CRC-986_100215-CW-T_CRC-986_300316-CW-T/mixture_chr'+chrom+'_CRC-986_100215-CW-T_'+str(st)+'x_CRC-986_300316-CW-T_'+str(sn)+'x',
                method, save=False)
            if method in ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan']:
                callmethod_snv.drop(['ANN'], axis=1, inplace=True)
            #print(callmethod_snv.shape)
            #print(list(callmethod_snv.columns))
            #print(callmethod_snv.head())
            callmethod_snv_list.append(callmethod_snv)
    callmethod_snv_allchrom = pd.concat(callmethod_snv_list)

    # callmethod_snv_allchrom = pd.concat([callmethod_snv_allchrom, calltablesseries['truth']], axis=1)
    callmethod_snv_allchrom = callmethod_snv_allchrom.join(calltablesseries[['truth']], how='outer')
    callmethod_snv_allchrom['truth'].fillna(False, inplace=True)
    callmethod_snv_allchrom['truth'].value_counts()
    #callmethod_snv_allchrom.head()
    
    features = list(callmethod_snv_allchrom.columns)
    print(features)
    for x in [method, method+'_score',  method+'_vaf', 'type', 'alt', 'chrom', 'pos', 'ref', method+'_totcov', method+'_altcov']:
        print(x)
        features.remove(x)
    print(features)

    #callmethod_snv_allchrom = pd.concat([callmethod_snv_allchrom, calltablesseries['truth']], axis=1)
    callmethod_snv_allchrom['truth'].fillna(False, inplace=True)
    callmethod_snv_allchrom['truth'].value_counts()

    callmethod_snv_allchrom[method].value_counts()
    callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] == 'PASS'] = 1
    callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] != 1] = 0

    forbiden_features = ['AF', 'OLD_VARIANT', 'TYPE', "cosmid_id", 'AN', "NS", 'CIGAR']
    score_features = ['ODDS', 'TLOD', 'SomaticEVS', 'SSF', 'SPV']
    print(features)

    Xlist = []
    for feature in features:
        print(feature)
        test = 0
        try:
            aux = callmethod_snv_allchrom[feature].fillna(0).astype(float)
            test = 1
        except:
            try: 
                aux = callmethod_snv_allchrom[feature].str.split(',').str[0].fillna(0).astype(float)
                test = 1
            except:    
                test = 0
        if test == 1 and feature != 'truth' and feature not in forbiden_features:
            Xlist.append(aux)
    X = pd.concat(Xlist, axis=1)
    y = callmethod_snv_allchrom['truth'].fillna(0)
    print(X.shape, y.shape)

    feature_name_dict = {}
    for i, x in enumerate(list(X.columns)):
        feature_name_dict[i] = x
    feature_name_dict

    model = RandomForestClassifier(random_state=0)
    gridsearchbestparams = pd.read_csv('data/featureanalysis/bestparams_'+method+'_986_150x.csv', index_col=0)
    gridsearchbestparams = gridsearchbestparams.to_dict()['0']
    for k, v in gridsearchbestparams.items():
        try:
            gridsearchbestparams[k] = int(v)
        except:
            print(k, v)
    model.set_params(**gridsearchbestparams) # retrive best hyper params
    # retrieve model
    import pickle
    # model = pickle.load(open("data/featureanalysis/model_"+method+"_986_150x.pkl", 'rb'))
    cv = StratifiedKFold(n_splits=10, shuffle = True, random_state=0) # prepare 10-fold CV
    cv_results = cross_validate(model, X, y, cv=cv, scoring=['roc_auc', 'average_precision'], return_estimator=True, verbose=6)

    # cv results
    print(cv_results)
    check = pd.read_csv('data/featureanalysis/cvresults_'+method+'_986_150x.csv', index_col=0)
    print(check)

    plt.rc('font', size=20) 
    fi_pd_list = []
    for idx, estimator in enumerate(cv_results['estimator']):
        fi_pd = pd.DataFrame(estimator.feature_importances_,
                             columns=['importance']).sort_values('importance', ascending=False)
        fi_pd_list.append(fi_pd)
    feature_importances = pd.concat(fi_pd_list, axis=1)
    feature_importances.columns = ['estimator_'+str(i) for i in range(10)]
    feature_importances.index = feature_name_dict.values()
    feature_importances.sort_values(by='estimator_0', ascending=False, inplace=True)
    feature_importances = feature_importances[(feature_importances.T != 0).any()]
    feature_importances['caller'] = method
    feature_importances['mean ROC AUC'] = np.mean(cv_results['test_roc_auc'])
    feature_importances['std ROC AUC'] = np.std(cv_results['test_roc_auc'])
    feature_importances_all.append(feature_importances)
    
feature_importances_all_pd = pd.concat(feature_importances_all)
#g = sns.catplot(data=feature_importances.T, kind="box", color=color_dict[method],
#                height=6, aspect=3.5);
#g.map_dataframe(sns.stripplot,
#                palette=["k"], dodge=True)
#g.set_xticklabels(rotation=45, ha='right')
#plt.xlabel('Features')
#plt.ylabel('Feature importance in Random Forest models in CV')
#plt.title(method + ' feature importance analysis. CV ROC AUC (acc {:.2f} +/- {:.2f})'.format(np.mean(cv_results['test_roc_auc']), np.std(cv_results['test_roc_auc'])))
#plt.savefig('data/featureanalysis/featureimportance_'+method+'_986_150x.svg')
#plt.savefig('data/featureanalysis/featureimportance_'+method+'_986_150x.png')
#plt.show()

In [ ]:
feature_importances_all_pd.to_csv('data/featureanalysis/featureimportance_allmethods_986_150x.csv')

In [ ]:
feature_importances_all_pd[feature_importances_all_pd['caller'] == method].T.iloc[:-2,:]

In [ ]:
a.shape[0]

In [ ]:
for method in config.methods:
    if method != 'smurf' and method != 'sinvict' and method != 'varnet':
        print(method)
        dg = feature_importances_all_pd[feature_importances_all_pd['caller'] == method].T.iloc[:-3,:]
        a = (dg.median() > 0.01)
        a = a[a == True]
        dg = dg[a.index]
        #maxdg = feature_importances_all_pd['caller'].value_counts().max()
        maxdg = 24
        cdg = a.shape[0]
        print(cdg, cdg/maxdg)
        if cdg/maxdg < 1:
            g = sns.catplot(data=dg, kind="box", color=color_dict[method], width=0.8, height=6, aspect=3.5*cdg/maxdg+0.5);
        else:
            g = sns.catplot(data=dg, kind="box", color=color_dict[method], width=0.8, height=6, aspect=3.5);
        g.map_dataframe(sns.stripplot, palette=["k"], dodge=True)
        g.set_xticklabels(rotation=45, ha='right')
        plt.xlabel('Features')
        plt.ylabel('Feature importance in Random Forest models in CV')
        plt.title(method + ' feature importance analysis. CV ROC AUC (acc {:.2f} +/- {:.2f})'.format(np.mean(cv_results['test_roc_auc']), np.std(cv_results['test_roc_auc'])))
        plt.ylim([0, dg.max().max()+0.01])
        plt.savefig('data/featureanalysis/featureimportance_'+method+'_986_150x.svg')
        plt.savefig('data/featureanalysis/featureimportance_'+method+'_986_150x.png')
        plt.show()

In [ ]:
np.random.seed(1234)
for method in [ 'abemus', 'varscan']: #config.methods_tissue: 'freebayes', 'mutect2', 'strelka2', 'vardict',
    # method = 'varnet'
    color_dict = {config.methods[i]: config.colors[i] for i in range(len(config.methods))}
    callmethod_snv_list = []
    for serie in [(70, 0), (70, 80), (50, 100), (30, 120), (20, 130), (10, 140), (5, 145)]:
        st, sn = serie
        #print(serie)
        for chrom in chroms:
            #print(chrom)
            callmethod_snv, _, _ = parse_caller_feature(
                'data/mixtures/mixtures_chr'+chrom+'/mixtures_chr'+chrom+'_CRC-986_100215-CW-T_CRC-986_300316-CW-T/mixture_chr'+chrom+'_CRC-986_100215-CW-T_'+str(st)+'x_CRC-986_300316-CW-T_'+str(sn)+'x',
                method, save=False)
            if method in ['freebayes', 'mutect2', 'strelka2', 'vardict', 'varscan']:
                callmethod_snv.drop(['ANN'], axis=1, inplace=True)
            #print(callmethod_snv.shape)
            #print(list(callmethod_snv.columns))
            #print(callmethod_snv.head())
            callmethod_snv_list.append(callmethod_snv)
    callmethod_snv_allchrom = pd.concat(callmethod_snv_list)

    # callmethod_snv_allchrom = pd.concat([callmethod_snv_allchrom, calltablesseries['truth']], axis=1)
    callmethod_snv_allchrom = callmethod_snv_allchrom.join(calltablesseries[['truth']], how='outer')
    callmethod_snv_allchrom['truth'].fillna(False, inplace=True)
    callmethod_snv_allchrom['truth'].value_counts()
    #callmethod_snv_allchrom.head()

    features = list(callmethod_snv_allchrom.columns)
    print(features)
    for x in [method, method+'_score',  method+'_vaf', 'type', 'alt', 'chrom', 'pos', 'ref', method+'_totcov', method+'_altcov']:
        print(x)
        features.remove(x)
    print(features)

    #callmethod_snv_allchrom = pd.concat([callmethod_snv_allchrom, calltablesseries['truth']], axis=1)
    callmethod_snv_allchrom['truth'].fillna(False, inplace=True)
    callmethod_snv_allchrom['truth'].value_counts()

    callmethod_snv_allchrom[method].value_counts()
    callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] == 'PASS'] = 1
    callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] != 1] = 0

    forbiden_features = ['AF', 'OLD_VARIANT', 'TYPE', "cosmid_id", 'AN', "NS", 'CIGAR']
    score_features = ['ODDS', 'TLOD', 'SomaticEVS', 'SSF', 'SPV']
    print(features)

    Xlist = []
    for feature in features:
        print(feature)
        test = 0
        try:
            aux = callmethod_snv_allchrom[feature].fillna(0).astype(float)
            test = 1
        except:
            try: 
                aux = callmethod_snv_allchrom[feature].str.split(',').str[0].fillna(0).astype(float)
                test = 1
            except:    
                test = 0
        if test == 1 and feature != 'truth' and feature not in forbiden_features:
            Xlist.append(aux)
    X = pd.concat(Xlist, axis=1)
    y = callmethod_snv_allchrom['truth'].fillna(0)
    print(X.shape, y.shape)

    feature_name_dict = {}
    for i, x in enumerate(list(X.columns)):
        feature_name_dict[i] = x
    feature_name_dict

    model = RandomForestClassifier(random_state=0)

    best_params = pd.read_csv('data/featureanalysis/bestparams_'+method+'_986_150x.csv', index_col=0).squeeze().to_dict()
    for k, v in best_params.items():
        try:
            best_params[k] = int(v)
        except:
            print(k, v)
    print(best_params)
    model = model.set_params(**best_params) # retrive best hyper params
    # save model
    import pickle
    #with open("data/featureanalysis/model_"+method+"_986_150x.pkl", "wb") as f:
    #    pickle.dump(model, f)
    # read pickled model
    cv = StratifiedKFold(n_splits=10, shuffle = True, random_state=0) # prepare 10-fold CV
    cv_results = cross_validate(model, X, y, cv=cv, scoring=['roc_auc', 'average_precision'], return_estimator=True, verbose=6)

    # save cv results
    cv_results_check = pd.read_csv('data/featureanalysis/cvresults_'+method+'_986_150x.csv', index_col=0)
    print(cv_results_check)

    plt.rc('font', size=20) 
    fi_pd_list = []
    for idx, estimator in enumerate(cv_results['estimator']):
        fi_pd = pd.DataFrame(estimator.feature_importances_,
                             columns=['importance']).sort_values('importance', ascending=False)
        fi_pd_list.append(fi_pd)
    feature_importances = pd.concat(fi_pd_list, axis=1)
    feature_importances.columns = ['estimator_'+str(i) for i in range(10)]
    feature_importances.index = feature_name_dict.values()
    feature_importances.sort_values(by='estimator_0', ascending=False, inplace=True)
    feature_importances = feature_importances[(feature_importances.T != 0).any()]
    print(round(100*feature_importances.T.median(), 2))
    g = sns.catplot(data=feature_importances.T, kind="box", color=color_dict[method],
                    height=6, aspect=3.5);
    g.map_dataframe(sns.stripplot,
                    palette=["k"], dodge=True)
    g.set_xticklabels(rotation=45, ha='right')
    plt.xlabel('Features')
    plt.ylabel('Feature importance in Random Forest models in CV')
    plt.title(method + ' feature importance analysis. CV ROC AUC (acc {:.2f} +/- {:.2f})'.format(np.mean(cv_results['test_roc_auc']), np.std(cv_results['test_roc_auc'])))
    plt.show()

In [ ]:
A = round(100*feature_importances.T.median(), 2)
A

In [ ]:
callmethod_snv_allchrom[method].value_counts()
callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] == 'PASS'] = 1
callmethod_snv_allchrom[method][callmethod_snv_allchrom[method] != 1] = 0

In [ ]:
forbiden_features = ['AF', 'OLD_VARIANT', 'TYPE', "cosmid_id", 'AN', "NS", 'CIGAR']
score_features = ['ODDS', 'TLOD', 'SomaticEVS', 'SSF', 'SPV']
print(features)

In [ ]:
plt.figure()
ax = plt.subplots(figsize=(10,10))
plt.rc('font', size=20) 
for feature in features:
    print(feature)
    test = 0
    try:
        pred = callmethod_snv_allchrom[feature].fillna(0).astype(float)
        test = 1
    except:
        try: 
            pred = callmethod_snv_allchrom[feature].str.split(',').str[0].fillna(0).astype(float)
            test = 1
        except:    
            test = 0
    if test == 1 and feature != 'truth' and feature not in forbiden_features:
        print('hello')
        #pred = callmethod_snv_allchrom[feature].fillna(0).astype(float)
        fpr, tpr, _ = roc_curve(callmethod_snv_allchrom['truth'].fillna(0), pred)
        if auc(fpr, tpr) < 0.5:
            print('invert')
            pred = -pred
        fpr, tpr, _ = roc_curve(callmethod_snv_allchrom['truth'].fillna(0), pred)
        print(auc(fpr, tpr))
        if auc(fpr, tpr) >= 0.9 and feature not in score_features:
            plot_roc_curve(fpr, tpr, estimator_name=feature, auc_score=auc(fpr, tpr), figax=ax, kwargs={'color':'r', 'lw': 3})
        if feature in score_features:
            plot_roc_curve(fpr, tpr, estimator_name=feature, auc_score=auc(fpr, tpr), figax=ax, kwargs={'color':'b', 'lw': 3})
        else:
            plot_roc_curve(fpr, tpr, estimator_name=None, auc_score=None, figax=ax, kwargs={'color':'grey', 'alpha': 0.5})
plt.legend(bbox_to_anchor=(1,1), loc="upper left")
plt.title('ROC curve of each feature of caller {}'.format(method))
#plt.grid()

In [ ]:
#plt.figure()
#ax = plt.subplots(figsize=(15,15))
#features = ['ODDS', 'AN']
##for feature in features:
#    precision, recall, _ = precision_recall_curve(concatres['truth'], concatres[feature])
#    #plt_pr_curve(fpr, tpr, estimator_name=feature, auc_score=auc(fpr, tpr), figax=ax)
#    plot_pr_curve(precision, recall, estimator_name=feature, f1_score=average_precision_score(concatres['truth'], concatres[feature]), figax=ax, kwargs={})
#    plt.ylim([-0.01,1.01])
    

plt.figure()
ax = plt.subplots(figsize=(15,15))
for feature in features:
    print(feature)
    test = 0
    try:
        pred = callmethod_snv_allchrom[feature].fillna(0).astype(float)
        test = 1
    except:
        test = 0
    if test == 1:
        print('hello')
        pred = callmethod_snv_allchrom[feature].fillna(0).astype(float)
        precision, recall, _ = precision_recall_curve(callmethod_snv_allchrom['freebayes'].fillna(0), pred)
        fpr, tpr, _ = roc_curve(callmethod_snv_allchrom['freebayes'].fillna(0), pred)
        if auc(fpr, tpr) < 0.5:
            print('invert')
            pred = -pred
        precision, recall, _ = precision_recall_curve(callmethod_snv_allchrom['freebayes'].fillna(0), pred)
        print(auc(fpr, tpr))
        if auc(fpr, tpr) >= 0.9 or feature == 'ODDS':
            plot_pr_curve(precision, recall, estimator_name=feature, f1_score=average_precision_score(callmethod_snv_allchrom['freebayes'].fillna(0), pred), figax=ax,  kwargs={'color':'r', 'lw': 3})
        else:
            plot_pr_curve(precision, recall, estimator_name=None, f1_score=None, figax=ax,  kwargs={'color':'grey', 'alpha': 0.5})
plt.ylim([-0.01,1.01])
plt.grid()

In [ ]:
average_precision_score(callmethod_snv_allchrom['truth'], callmethod_snv_allchrom['freebayes_score'].fillna(0).astype(float))